Follows $\texttt{hls4ml/test/pytest/test\_extensions.py}$, but without the optimization stuff

In [1]:
import os
from pathlib import Path

import numpy as np
import pytest
import tensorflow as tf

import hls4ml

test_root_path = Path(os.getcwd())

2024-10-10 13:43:50.834149: I tensorflow/core/platform/cpu_feature_guard.cc:193] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  AVX2 AVX_VNNI FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags.
2024-10-10 13:43:50.949395: I tensorflow/core/util/port.cc:104] oneDNN custom operations are on. You may see slightly different numerical results due to floating-point round-off errors from different computation orders. To turn them off, set the environment variable `TF_ENABLE_ONEDNN_OPTS=0`.
2024-10-10 13:43:50.953763: W tensorflow/compiler/xla/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libcudart.so.11.0'; dlerror: libcudart.so.11.0: cannot open shared object file: No such file or directory
2024-10-10 13:43:50.953787: I tensorflow/compiler/xla/stream_executor/cuda/cudart_stub.cc:29] Ignore above cudar

/home/aelabd/RHEED/hls4ml/hls4ml/converters/__init__.py:29: UserWarning: WARNING: Pytorch converter is not enabled!
  warnings.warn("WARNING: Pytorch converter is not enabled!", stacklevel=1)


In [2]:
# Keras implementation of a custom layer
class KReverse(tf.keras.layers.Layer):
    '''Keras implementation of a hypothetical custom layer'''

    def __init__(self):
        super().__init__()

    def call(self, inputs):
        return tf.reverse(inputs, axis=[-1])

    def get_config(self):
        # Breaks serialization and parsing in hls4ml if not defined
        return super().get_config()


# hls4ml layer implementation
class HReverse(hls4ml.model.layers.Layer):
    '''hls4ml implementation of a hypothetical custom layer'''

    def initialize(self):
        inp = self.get_input_variable()
        shape = inp.shape
        dims = inp.dim_names
        self.add_output_variable(shape, dims)

# Parser for converter
def parse_reverse_layer(keras_layer, input_names, input_shapes, data_reader):
    layer = {}
    layer['class_name'] = 'HReverse'
    layer['name'] = keras_layer['config']['name']
    layer['n_in'] = input_shapes[0][1]

    if input_names is not None:
        layer['inputs'] = input_names

    return layer, [shape for shape in input_shapes[0]]

# HLS Templates - No specific pragmas used; generic enough for both Intel and Vivado
rev_config_template = """struct config{index} : nnet::reverse_config {{
    static const unsigned n_in = {n_in};
}};\n"""

rev_function_template = 'nnet::reverse<{input_t}, {config}>({input}, {output});'
rev_include_list = ['nnet_utils/nnet_reverse.h']

class HReverseConfigTemplate(hls4ml.backends.template.LayerConfigTemplate):
    def __init__(self):
        super().__init__(HReverse)
        self.template = rev_config_template

    def format(self, node):
        params = self._default_config_params(node)
        return self.template.format(**params)
    
class HReverseFunctionTemplate(hls4ml.backends.template.FunctionCallTemplate):
    def __init__(self):
        super().__init__(HReverse, include_header=rev_include_list)
        self.template = rev_function_template

    def format(self, node):
        params = self._default_function_params(node)
        return self.template.format(**params)

In [3]:
rev_hls = """#ifndef NNET_REVERSE_H_
#define NNET_REVERSE_H_

#include "nnet_common.h"

namespace nnet {

struct reverse_config {
    static const unsigned n_in = 10;
};

template<class data_T, typename CONFIG_T>
void reverse(
    data_T input[CONFIG_T::n_in],
    data_T reversed[CONFIG_T::n_in]
) {
    for (int i = 0; i < CONFIG_T::n_in; i++) {
        reversed[CONFIG_T::n_in - 1 - i] = input[i];
    }
}

}

#endif
"""


In [4]:
# @pytest.fixture(scope='session', autouse=True)
def register_custom_layer():
    # Register the converter for custom Keras layer
    hls4ml.converters.register_keras_layer_handler('KReverse', parse_reverse_layer)

    # Register the hls4ml's IR layer
    hls4ml.model.layers.register_layer('HReverse', HReverse)

@pytest.mark.parametrize('backend_id', ['Vivado', 'Vitis', 'Quartus'])
def test_extensions(tmp_path, backend_id):
    
    # Register the optimization passes (if any)
    backend = hls4ml.backends.get_backend(backend_id)
    # ip_flow = hls4ml.model.flow.get_flow(backend.get_default_flow())
    # Add the pass into the main optimization flow
    # optimize_flow = [flow for flow in ip_flow.requires if ':optimize' in flow][0]
    # optmizer_name = f'{backend_id.lower()}:remove_duplicate_reverse'
    

    # Register template passes for the given backend
    try:
        register_custom_layer()
        backend.register_template(HReverseConfigTemplate)
        backend.register_template(HReverseFunctionTemplate)
        print("Registered custom layer")
    except Exception:
        pass

    # Register HLS implementation
    p = tmp_path / 'nnet_reverse.h'
    p.write_text(rev_hls)
    backend.register_source(p)

    # Test if it works
    kmodel = tf.keras.models.Sequential(
        [
            tf.keras.layers.Input(shape=(8,)),
            KReverse(),
            tf.keras.layers.ReLU(),
            # These two should be removed by the optimizer
            KReverse(),
            KReverse(),
        ]
    )

    x = np.random.randint(-5, 5, (8,), dtype='int32')
    kres = kmodel(x)

    hmodel = hls4ml.converters.convert_from_keras_model(
        kmodel,
        output_dir=str(test_root_path / f'hls4mlprj_extensions_{backend_id}'),
        backend=backend_id,
        io_type='io_parallel',
        hls_config={'Model': {'Precision': 'ap_int<6>', 'ReuseFactor': 1}},
    )

    hmodel.compile()
    hres = hmodel.predict(x.astype('float32'))

    np.testing.assert_array_equal(kres, hres)
    return x, kres, hres

In [5]:
x, kres, hres = test_extensions(Path(os.path.join(os.getcwd(),'tmp')), 'Vivado')

Registered custom layer and optimizer
Interpreting Sequential
Topology:
Layer name: input_1, layer type: InputLayer, input shapes: [[None, 8]], output shape: [None, 8]
Layer name: k_reverse, layer type: HReverse, input shapes: [[None, 8]], output shape: [None, 8]
Layer name: re_lu, layer type: Activation, input shapes: [[None, 8]], output shape: [None, 8]
Layer name: k_reverse_1, layer type: HReverse, input shapes: [[None, 8]], output shape: [None, 8]
Layer name: k_reverse_2, layer type: HReverse, input shapes: [[None, 8]], output shape: [None, 8]
Creating HLS model


shape: [8], 
dims: ['N_INPUT_1_1']




shape: [8], 
dims: ['N_INPUT_1_1']




shape: [8], 
dims: ['N_INPUT_1_1']


Writing HLS project


2024-10-10 13:43:54.308857: I tensorflow/compiler/xla/stream_executor/cuda/cuda_gpu_executor.cc:967] could not open file to read NUMA node: /sys/bus/pci/devices/0000:01:00.0/numa_node
Your kernel may have been built without NUMA support.
2024-10-10 13:43:54.309094: W tensorflow/compiler/xla/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libcudart.so.11.0'; dlerror: libcudart.so.11.0: cannot open shared object file: No such file or directory
2024-10-10 13:43:54.309207: W tensorflow/compiler/xla/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libcublas.so.11'; dlerror: libcublas.so.11: cannot open shared object file: No such file or directory
2024-10-10 13:43:54.309278: W tensorflow/compiler/xla/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libcublasLt.so.11'; dlerror: libcublasLt.so.11: cannot open shared object file: No such file or directory
2024-10-10 13:43:54.309346: W tensorf

Done
